In [ ]:
!nvidia-smi

Googleドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

デバイスの設定

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('使用されるデバイス：', device)

In [ ]:
data_dir = '/content/drive/MyDrive/work/data/'
weight_dir = '/content/drive/MyDrive/work/weights/'
log_dir = '/content/drive/MyDrive/work/logs/'

データローダーの作成

In [ ]:
import torch.utils.data as data
from kinoko_takenoko import make_filepath_list, GetBBoxAndLabel, DataTransform, PreprocessKinokoTakenoko, multiobject_collate_fn

In [ ]:
train_img_list, train_anno_list, val_img_list, val_anno_list = make_filepath_list(data_dir)

In [ ]:
len(train_img_list), len(train_anno_list), len(val_img_list), len(val_anno_list)

In [ ]:
labels = ["kinoko", "takenoko"]
color_mean = [164, 172, 176]
input_size = 300

In [ ]:
train_dataset = PreprocessKinokoTakenoko(
    train_img_list,
    train_anno_list,
    phase="train",
    transform=DataTransform(input_size, color_mean),
    get_bbox_label=GetBBoxAndLabel(labels))

val_dataset = PreprocessKinokoTakenoko(
    val_img_list,
    val_anno_list,
    phase="val",
    transform=DataTransform(input_size, color_mean),
    get_bbox_label=GetBBoxAndLabel(labels))

In [ ]:
batch_size = 16

train_dataloader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=multiobject_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=multiobject_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

SSDモデルの作成

In [ ]:
!pip install torchinfo

In [ ]:
import torch.nn as nn
import torch.nn.init as init
from torchinfo import summary
from ssd import SSD

In [ ]:
ssd_cfg = {
    'classes_num': 3,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'dbox_num': [4, 6, 6, 6, 4, 4],  # DBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}
net = SSD(phase='train', cfg=ssd_cfg)
vgg_weights = torch.load(weight_dir + 'vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights) 

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

net.extras.apply(weights_init) # extrasネットワーク
net.loc.apply(weights_init)    # locネットワーク
net.conf.apply(weights_init)   #confネットワーク

summary(
    net,
    input_size=(batch_size, 3, 300, 300),
    col_names=['input_size','output_size', 'num_params'])

損失関数とオプティマイザーの作成

In [ ]:
import torch.optim as optim
from ssd import MultiBoxLoss

In [ ]:
criterion = MultiBoxLoss(
    jaccard_thresh=0.5, # 背景のDBoxに分類するときのIoUの閾値
    neg_pos=3, # 背景のDBoxの数はPositive DBoxの何倍にするか
    device=device) # ネットワークのTensorに割り当てるデバイス

optimizer = optim.SGD(
    net.parameters(),  # SSDモデルのパラメーター
    lr=1e-3,           # 学習率
    momentum=0.9,      # 慣性項に割り当てる係数
    weight_decay=5e-4) # 重み更新時のL2正則化の係数

SSDモデルの学習(パラメーターの更新)を行う関数

In [ ]:
import time
import pandas as pd

def train(net, dataloaders_dict, criterion, optimizer, num_epochs):
    '''
    Parameters:
        net(object): SSDモデル
        dataloaders_dict(objectのdict): データーローダー
        criterion(object): 損失関数
        optimizer(object): オプティマイザー
        num_epochs(object): 学習回数
    '''
    # SSDモデルでGPUを使用
    net.to(device)

    # ネットワークの構成に対して最適なアルゴリズムを見つけて高速化させる
    torch.backends.cudnn.benchmark = True

    iteration = 1          # イテレーション(ステップ)カウンター
    epoch_train_loss = 0.0 # 訓練1エポックごとの損失和
    epoch_val_loss = 0.0   # 検証1エポックごとの損失和
    logs = []              # 損失のログを記録するリスト

    # 学習、または検証のエポックごとのループ
    for epoch in range(num_epochs):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        # 現在のエポック数を出力
        print('---------------------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('---------------------------------------')

        # エポック10回につき検証を1回行う
        for phase in ['train', 'val']:
            # エポックが10回に達するまではモデルを訓練モードにする
            if phase == 'train':
                net.train()  # モデルを訓練モードにする
            else:
                # エポックが10回に達していたらモデルを検証モードにして検証開始
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードにする
                    print('---------------------------------------')
                    print('（validation）')
                else:
                    # 10回に達していなければ次のエポックに進む
                    continue

            # 1ステップにおけるミニバッチを使用した学習または検証
            # データローダーをイテレートしてミニバッチを抽出
            for images, targets in dataloaders_dict[phase]:
                # 画像データにデバイスを割り当てる
                images = images.to(device)
                # 教師データ(正解BBoxのアノテーション情報)
                # (バッチサイズ, 物体数, 5[xmin, ymin, xmax, ymax, label_index])
                # にデバイスを割り当てる
                targets = [ann.to(device) for ann in targets]

                # optimizerが保持する勾配を0で初期化(累積しないように)
                optimizer.zero_grad()

                # 順伝搬（forward）とバックプロパゲーション(訓練時のみ)
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）を行って(loc, conf, dbox_list)を取得
                    # ・locの出力(バッチサイズ, 8732, 4[Δcx, Δcy, Δw, Δh])
                    # ・confの出力(バッチサイズ, 8732, 21)
                    # ・DBoxの情報(8732, 4[cx, cy, width, height])
                    outputs = net(images)

                    # Positive DBoxのオフセット情報の損失平均
                    # ミニバッチにおけるPositive DBoxの確信度の損失平均
                    loss_l, loss_c = criterion(outputs, targets)
                    # 2つの損失を合計する
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーションによるパラメーター更新を行う
                    if phase == 'train':
                        loss.backward()  # バックプロパゲーション

                        # 勾配が大きすぎると不安定になるので
                        # clipで勾配の上限を2.0に制限する
                        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                        # 勾配降下法の更新式を適用してバイアス、重みを更新
                        optimizer.step()

                        # ミニバッチを10個処理(10ステップ)ごとに損失を出力
                        if (iteration % 10 == 0):
                            # 10ステップの所要時間を取得
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            # ステップ数、損失、所要時間を出力
                            print('ステップ( {} )  loss: {:.4f} -- time: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        # エポックの損失をepoch_train_lossに加算する
                        epoch_train_loss += loss.item()
                        # ステップ数を1増やす
                        iteration += 1

                    # 検証モードでは順伝播後の損失の記録のみを行う
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        # エポック終了時の時刻を取得
        t_epoch_finish = time.time()
        print('---------------------------------------')
        # 訓練データの損失と検証データの損失を出力
        print('train_loss: {:.4f} - val_loss(Every 10 epochs): {:.4f}'.format(
            epoch_train_loss, epoch_val_loss))
        # エポック終了までに要した時間を取得
        print('time:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        # 次のエポックの開始時刻を取得
        t_epoch_start = time.time()

        # エポックごとに損失をdictオブジェクトに保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss,
                     'val_loss': epoch_val_loss}
        # ログのリストに追加
        logs.append(log_epoch)
        # ログのリストをデータフレームに変換
        df = pd.DataFrame(logs)
        # ログファイルに保存
        df.to_csv(log_dir + 'epoch_loss.csv')

        # 訓練時の損失和を0で初期化
        epoch_train_loss = 0.0
        # 検証時の損失和を0で初期化
        epoch_val_loss = 0.0

        # 1エポック終了ごとにモデルのパラメーター値を保存
        if ((epoch+1) % 10 == 0):
            torch.save(
                net.state_dict(),
                weight_dir + 'kinotake_ssd_weights' + str(epoch+1) + '.pth'
            )
            print('--saved weights--')

学習と検証を実行

In [ ]:
num_epochs= 500

train(net,                   # SSDモデル
      dataloaders_dict,      # データローダー
      criterion,             # 損失関数
      optimizer,             # オプティマイザー
      num_epochs=num_epochs) # エポック数